# Import Libraries

In [2]:
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [26]:
import os
import numpy as np
import contractions as ctc
import re
import pandas as pd

In [4]:
from tensorflow.keras import models,layers
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

In [94]:
import tensorflow_hub as hub
from sklearn.metrics import classification_report

# Loading google's pretrained Universal sentence encoder model


In [7]:
embed = load_model("./model/universal-sentence-encoder_4")


2021-09-26 22:39:04.104079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-09-26 22:39:04.104115: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-09-26 22:39:04.104715: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-26 22:39:06.368487: I tensorflow/c

In [13]:
tf.keras.utils.model_to_dot(embed,show_shapes=True,expand_nested=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [22]:

embedding = embed(["The quick brown fox jumps over the lazy dog. Who are you?"])

In [23]:
embedding.shape

TensorShape([1, 512])

# Training and Testing on BBC Dataset

In [27]:
df=pd.read_csv("./TrainData.csv")

In [28]:
df.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [30]:
df.Category.unique()

array(['business', 'tech', 'politics', 'sport', 'entertainment'],
      dtype=object)

In [31]:
labels={'business':0, 'tech':1, 'politics':2, 'sport':3, 'entertainment':4}

In [32]:
df["labels"]=df["Category"].map(labels)

In [33]:
df.head()

,Text,Category,labels
0,worldcom ex-boss launches defence lawyers defe...,business,0
1,german business confidence slides german busin...,business,0
2,bbc poll indicates economic gloom citizens in ...,business,0
3,lifestyle governs mobile choice faster bett...,tech,1
4,enron bosses in $168m payout eighteen former e...,business,0


In [34]:
x=embed(df.Text)

In [35]:
x.shape

TensorShape([1490, 512])

In [46]:
model=Sequential()
model.add(InputLayer((512)))
model.add(Dense(256,activation="relu"))
model.add(Dense(5,activation="softmax"))

In [47]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 1285      
Total params: 132,613
Trainable params: 132,613
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [49]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,mode='min')

In [50]:
model.fit(x,df.labels.values,epochs=10,batch_size=16,validation_split=0.1,callbacks=[es])

Epoch 1/10
84/84 [==============================] - 1s 3ms/step - loss: 0.7587 - accuracy: 0.8650 - val_loss: 0.2453 - val_accuracy: 0.9732
Epoch 2/10
84/84 [==============================] - 0s 2ms/step - loss: 0.1383 - accuracy: 0.9739 - val_loss: 0.1392 - val_accuracy: 0.9664
Epoch 3/10
84/84 [==============================] - 0s 2ms/step - loss: 0.0752 - accuracy: 0.9851 - val_loss: 0.1141 - val_accuracy: 0.9732
Epoch 4/10
84/84 [==============================] - 0s 2ms/step - loss: 0.0503 - accuracy: 0.9881 - val_loss: 0.1010 - val_accuracy: 0.9732
Epoch 5/10
84/84 [==============================] - 0s 2ms/step - loss: 0.0345 - accuracy: 0.9948 - val_loss: 0.1073 - val_accuracy: 0.9799
Epoch 6/10
84/84 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 0.9985 - val_loss: 0.1008 - val_accuracy: 0.9732
Epoch 7/10
84/84 [==============================] - 0s 2ms/step - loss: 0.0178 - accuracy: 1.0000 - val_loss: 0.1015 - val_accuracy: 0.9732
Epoch 8/10
84/84 [==

In [51]:
df_test=pd.read_csv("./TestData_Inputs.csv")

In [52]:
df_test.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [53]:
df_label=pd.read_csv("./TestLabels.csv")

In [55]:
df_label.head()

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,sport
3,459,business
4,1020,sport


In [56]:
df_label["labels"]=df_label["Category"].map(labels)

In [57]:
df_label.head()

,ArticleId,Category,labels
0,1018,sport,3
1,1319,tech,1
2,1138,sport,3
3,459,business,0
4,1020,sport,3


In [58]:
x_test=embed(df_test.Text)

In [60]:
model.evaluate(x_test,df_label.labels.values)

23/23 [==============================] - 0s 766us/step - loss: 0.0994 - accuracy: 0.9714


[0.09941919893026352, 0.9714285731315613]

In [99]:
y_pred=model.predict(x_test)

In [100]:
y_pred=np.argmax(y_pred,axis=1)

In [101]:
rp=classification_report(df_label.labels.values,y_pred)

In [102]:
print(rp)

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       176
           1       0.96      0.99      0.97       135
           2       0.97      0.97      0.97       143
           3       1.00      0.98      0.99       168
           4       0.98      0.97      0.98       113

    accuracy                           0.97       735
   macro avg       0.97      0.97      0.97       735
weighted avg       0.97      0.97      0.97       735



# Training and Testing on IMDB dataset

In [61]:
df_imdb=pd.read_csv("IMDB Dataset.csv")

In [62]:
df_imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [63]:
labels2={"positive":1,"negative":0}

In [65]:
df_imdb["lables"]=df_imdb["sentiment"].map(labels2)

In [66]:
df_imdb.head()

,review,sentiment,lables
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [67]:
df_train1=df_imdb.iloc[0:25000]
df_test1=df_imdb.iloc[25000:]

In [68]:
df_train1.shape

(25000, 3)

In [69]:
df_test1.shape

(25000, 3)

In [72]:
x_1=embed(df_train1.review)

In [85]:
model1=Sequential()
model1.add(InputLayer((512)))
model1.add(Dense(256,activation="relu"))
model1.add(Dense(5,activation="softmax"))

In [86]:
model1.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [87]:
model1.fit(x_1,df_train1.lables.values,epochs=10,batch_size=16,validation_split=0.1,callbacks=[es])

Epoch 1/10
1407/1407 [==============================] - 2s 1ms/step - loss: 0.3614 - accuracy: 0.8461 - val_loss: 0.3604 - val_accuracy: 0.8436
Epoch 2/10
1407/1407 [==============================] - 2s 1ms/step - loss: 0.3227 - accuracy: 0.8605 - val_loss: 0.3547 - val_accuracy: 0.8456
Epoch 3/10
1407/1407 [==============================] - 2s 1ms/step - loss: 0.3104 - accuracy: 0.8661 - val_loss: 0.3603 - val_accuracy: 0.8376
Epoch 4/10
1407/1407 [==============================] - 2s 1ms/step - loss: 0.2931 - accuracy: 0.8723 - val_loss: 0.3424 - val_accuracy: 0.8452
Epoch 5/10
1407/1407 [==============================] - 2s 1ms/step - loss: 0.2756 - accuracy: 0.8834 - val_loss: 0.3442 - val_accuracy: 0.8476
Epoch 6/10
1407/1407 [==============================] - 2s 1ms/step - loss: 0.2548 - accuracy: 0.8931 - val_loss: 0.3358 - val_accuracy: 0.8528
Epoch 7/10
1407/1407 [==============================] - 2s 1ms/step - loss: 0.2348 - accuracy: 0.9037 - val_loss: 0.3566 - val_accuracy:

In [78]:
x_test1=embed(df_test1.review)

In [88]:
model1.evaluate(x_test1,df_test1.lables.values)

782/782 [==============================] - 1s 836us/step - loss: 0.3273 - accuracy: 0.8656


[0.3273104131221771, 0.8655999898910522]

In [95]:
y_pred=model1.predict(x_test1)

In [97]:
y_pred=np.argmax(y_pred,axis=1)

In [98]:
rp=classification_report(df_test1.lables.values,y_pred)
print(rp)

              precision    recall  f1-score   support

           0       0.88      0.84      0.86     12474
           1       0.85      0.89      0.87     12526

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000

